In [11]:
!pip install -q -U kaggle_environments

     |████████████████████████████████| 1.9 MB 5.2 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 


In [20]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

#1. Агент, выкидывающий "только камень"

In [13]:
%%writefile rock_agent.py

#0 - rock
#1 - paper
#2 - scissors
def rock_agent(observation, configuration):
    return 0

Writing rock_agent.py


In [21]:
evaluate(
    "rps", #environment to use - no need to change
    ["rock_agent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[-98.0, 98.0]]

#2. Агент, выкидывающий "только бумагу"

In [23]:
%%writefile paper_agent.py

def paper_agent(observation, configuration):
    return 1

Writing paper_agent.py


In [24]:
evaluate(
    "rps", #environment to use - no need to change
    ["paper_agent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[-97.0, 97.0]]

#3. Агент, выкидывающий "только ножницы"

In [25]:
%%writefile scissors_agent.py

def scissors_agent(observation, configuration):
    return 2

Writing scissors_agent.py


In [26]:
evaluate(
    "rps", #environment to use - no need to change
    ["scissors_agent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[-99.0, 99.0]]

# 4. Агент, выкидывающий "только не камень"

In [55]:
%%writefile not_rock.py
import random

def not_rock(observation, configuration):
    return random.randrange(1, configuration.signs)

Overwriting not_rock.py


In [56]:
evaluate(
    "rps", #environment to use - no need to change
    ["not_rock.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[0, 0]]

# 5. Агент, выкидывающий "только не бумага"

In [53]:
%%writefile not_paper.py
import random

def not_paper(observation, configuration):
    return random.randrange(0, configuration.signs, 2)

Overwriting not_paper.py


In [54]:
evaluate(
    "rps", #environment to use - no need to change
    ["not_paper.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[0, 0]]

# 6. Агент, выкидывающий "только не ножницы"

In [51]:
%%writefile not_scissors.py
import random

def not_scissors(observation, configuration):
    return random.randrange(0, configuration.signs - 1)

Overwriting not_scissors.py


In [52]:
evaluate(
    "rps", #environment to use - no need to change
    ["not_scissors.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[-40.0, 40.0]]

#7. Агент, выкидывающий повторную комбинацию

In [33]:
%%writefile copy_opponent.py

import random

def copy_opponent(observation, configuration):
    if observation.step > 0:
        return observation.lastOpponentAction
    else:
        return random.randrange(0, configuration.signs)

Writing copy_opponent.py


In [34]:
evaluate(
    "rps", #environment to use - no need to change
    ["copy_opponent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[-22.0, 22.0]]

# 8. Агент, выкидывающий случайную комбинацию

In [35]:
%%writefile random_opponent.py

import random

def random_opponent(observation, configuration):
    return random.randrange(0, configuration.signs)

Writing random_opponent.py


In [36]:
evaluate(
    "rps", #environment to use - no need to change
    ["random_opponent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[0, 0]]

# 9. Агент, который прибавляет к последней комбинации +1 в случае проигрыша в предыдущей игровой сессии

In [49]:
%%writefile lose_agent.py

import random

last_lose_action = None
def lose_agent(observation, configuration):
    global last_lose_action
    if observation.step == 0:
        last_lose_action = random.randrange(0, configuration.signs)
    elif (observation.lastOpponentAction - last_lose_action) in [-2, 0, 1]:
        last_lose_action = (observation.lastOpponentAction + 1) % configuration.signs
    return last_lose_action



Overwriting lose_agent.py


In [50]:
evaluate(
    "rps", #environment to use - no need to change
    ["lose_agent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[49.0, -49.0]]

# 10. Агент, выкидывающий все комбинации по порядку

In [3]:
%%writefile sequence_agent.py

counter = -1

def sequence(observation, configuration):
    global counter
    counter += 1
    if counter % 3 == 0:
        return 0
    elif counter % 3 == 1:
        return 1
    else:
        return 2

Writing sequence_agent.py


In [42]:
evaluate(
    "rps", #environment to use - no need to change
    ["sequence_agent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[0, 0]]

# 11. Агент, выкидывающий повторную кобинацию в случае выигрыаша, и меняет на случайную в случае проигрыша

In [47]:
%%writefile win_reaction.py

import random
last_action = None
def win_reaction(observation, configuration):
    global last_action
    if observation.step == 0:
        last_action = random.randrange(0, configuration.signs)
    elif (observation.lastOpponentAction - last_action) in [-2, 0, 1]:
        last_action = random.randrange(0, configuration.signs)
    else:
        return last_action

Overwriting win_reaction.py


In [48]:
evaluate(
    "rps", #environment to use - no need to change
    ["win_reaction.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[36.0, -36.0]]

# 12. Агент, выкидывающий в случае выигрыша "только камень"

In [45]:
%%writefile rock_if_win.py

import random
last_win_action = None
def rock_if_win(observation, configuration):
    global last_win_action
    if observation.step == 0:
       last_win_action = random.randrange(0, configuration.signs)
    elif (observation.lastOpponentAction - last_win_action) in [-1, 2]:
        return 0
    else:
        last_win_action = random.randrange(0, configuration.signs)

Overwriting rock_if_win.py


In [46]:
evaluate(
    "rps", #environment to use - no need to change
    ["rock_if_win.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[-98.0, 98.0]]

In [57]:
agents = ['rock_agent.py',
          'paper_agent.py',
          'scissors_agent.py',
          'not_rock.py',
          'not_paper.py',
          'not_scissors.py',
          'copy_opponent.py',
          'random_opponent.py', 
          'lose_agent.py',
          'sequence_agent.py',
          'win_reaction.py',
          'rock_if_win.py']


rank = {agent: 0 for agent in agents}

for times in range(3):
          
    for i in range(len(agents)):
        for j in range(i+1, len(agents)):
            e = evaluate(
            "rps", #environment to use - no need to change
            [agents[i], agents[j]], #agents to evaluate
            configuration={"episodeSteps": 100, 'tieRewardThreshold': 1} #number of episodes 
            )
            if e[0][0] > e[0][1]:
                rank[agents[i]] += 1
            elif e[0][0] < e[0][1]:
                rank[agents[j]] += 1

In [58]:
rank_se = pd.Series(rank, name="Рейтинг турнира по количеству выигранных партий").sort_values(ascending=False).to_frame()
rank_se

,Рейтинг турнира по количеству выигранных партий
lose_agent.py,28
win_reaction.py,22
not_scissors.py,19
paper_agent.py,14
random_opponent.py,14
not_rock.py,13
not_paper.py,13
copy_opponent.py,12
rock_if_win.py,12
sequence_agent.py,10
